# Hybrid Model Testing Script
This notebook loads the trained models and generates movie recommendations using both content-based and collaborative filtering methods.

## Step 1: Import Libraries
To use the saved models and process user input, we need essential libraries for vectorization, similarity computation, and rating prediction.

### What it Does
- Loads pandas for data handling.
- Loads joblib for loading model files.
- Imports cosine_similarity for manual comparison if needed.
- Loads the Surprise SVD model for collaborative filtering.

In [3]:
import pandas as pd
import joblib
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD

## Step 2: Load Trained Models and Dataset
This block loads pre-trained models and the cleaned movie dataset, so we can perform real-world tests on it.

### What it Does
- Loads TF-IDF vectorizer.
- Loads the NearestNeighbors index matrix.
- Loads the SVD collaborative filtering model.
- Loads the processed dataset with genres, ratings, and titles.

### Variables
- `tfidf_vectorizer`: Used to convert genres to numerical vectors.
- `neighbor_indices`: Stores top 50 similar movies for each.
- `svd_model`: Predicts user-movie rating.
- `movies_df`: DataFrame containing movie metadata.

In [6]:
tfidf_vectorizer = joblib.load("models/tfidf_vectorizer.pkl")
neighbor_indices = joblib.load("models/nearest_neighbors_indices.pkl")
svd_model = joblib.load("models/svd_model.pkl")
movies_df = pd.read_csv("cleaned_imdb_movies.csv")

## Step 3: Define Title Matching Helper
User input might not match movie titles exactly. A search function increases usability and robustness.

### What it Does
- Accepts user query (partial title).
- Returns the first matching movie.

### Variables
- `query`: User input string.
- `match`: First title match from `movies_df`.

In [9]:
def find_title_match(query):
    matches = movies_df[movies_df["primaryTitle"].str.contains(query, case=False, na=False)]
    if matches.empty:
        return None
    return matches.iloc[0]

## Step 4: Recommend Movies

This function combines content-based and collaborative filtering to produce recommendations that consider genre similarity and rating predictions.

### What it Does
- Finds the index of the matched movie.
- Uses neighbor index to find similar movies.
- Predicts user rating for each similar movie using SVD.
- Scores results by combining actual and predicted ratings.

### Variables
- `title`: Query title.
- `movie_idx`: Index of matched movie.
- `similar_indices`: Top-k neighbors.
- `results_df`: DataFrame of recommended results.

In [12]:
def recommend_movies(title, user_id=0, top_n=10):
    match = find_title_match(title)
    if match is None:
        print("Movie not found.")
        return
    
    movie_idx = match.name
    similar_indices = neighbor_indices[movie_idx][1:]
    similar_movies = movies_df.iloc[similar_indices][["primaryTitle", "genres", "averageRating"]].copy()

    similar_movies["PredictedRating"] = similar_movies["primaryTitle"].apply(
        lambda x: svd_model.predict(user_id, x).est
    )
    similar_movies["score"] = (similar_movies["averageRating"] + similar_movies["PredictedRating"]) / 2
    similar_movies = similar_movies.sort_values("score", ascending=False).head(top_n)

    print(f"Top {top_n} recommendations for '{match['primaryTitle']}':\n")
    print(similar_movies[["primaryTitle", "genres", "averageRating", "PredictedRating", "score"]])

## Step 5: Run Test

Simulates a recommendation call from user input.

### What it Does
- Asks for movie title.
- Displays top 10 recommendations using both models.

In [33]:
recommend_movies("lord of the rings")

Top 10 recommendations for 'The Lord of the Rings: The Fellowship of the Ring':

                          primaryTitle                   genres  \
306                           Big Fish  Adventure,Drama,Fantasy   
2444         The City of Lost Children  Adventure,Drama,Fantasy   
2509                         Excalibur  Adventure,Drama,Fantasy   
37929                  The Fateful Day  Adventure,Drama,Fantasy   
2165       Beasts of the Southern Wild  Adventure,Drama,Fantasy   
30095                  City of Pirates  Adventure,Drama,Fantasy   
9844                        Freaks Out  Adventure,Drama,Fantasy   
7592   Valerie and Her Week of Wonders  Adventure,Drama,Fantasy   
38698               Lost and Beautiful  Adventure,Drama,Fantasy   
81532            Eréndira la indomable  Adventure,Drama,Fantasy   

       averageRating  PredictedRating     score  
306              8.0         6.079076  7.039538  
2444             7.5         6.256647  6.878323  
2509             7.3         6.